<a href="https://colab.research.google.com/github/iviecomarti/test-stardist-onnx-pytorch-qupath/blob/main/convert_tf2onnx/convert_stardist_tf2onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook to convert StarDist models from TensorFlow SavedModelBundle to ONNX

First, we need to install the required dependencies

If you use these models in a publication, please cite the original authors as indicated here: https://github.com/stardist/stardist



In [ ]:
!pip -q uninstall -y numpy

In [ ]:
!pip -q install "numpy<2" "tf2onnx==1.16.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grain 0.2.15 requires protobuf>=5.28.3, but you have protobuf 3.20.3 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opentelemetry-proto 1.38.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
opencv-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires nump

## Fetch the models from StarDist-ImageJ


In [ ]:
BASE = "https://raw.githubusercontent.com/stardist/stardist-imagej/master/src/main/resources/models/2D"

ZIPS = [
    "he_heavy_augment.zip",
    "dsb2018_heavy_augment.zip",
]

for z in ZIPS:
    !wget -q -O "{z}" "{BASE}/{z}"
    print("Downloaded", z)



Downloaded he_heavy_augment.zip
Downloaded dsb2018_heavy_augment.zip


In [ ]:
import os

for z in ZIPS:
    folder = z.replace(".zip", "")
    !rm -rf "{folder}"
    !mkdir -p "{folder}"
    !unzip -q "{z}" -d "{folder}"


## Convert the models to ONNX

In [ ]:
!mkdir -p "stardist_onnx_models"
!python -m tf2onnx.convert --saved-model "/content/he_heavy_augment" --opset 13 --output "/content/stardist_onnx_models/he_heavy_augment.onnx"
!python -m tf2onnx.convert --saved-model "/content/dsb2018_heavy_augment" --opset 13 --output "/content/stardist_onnx_models/dsb2018_heavy_augment.onnx"


2026-02-14 12:05:02.458753: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-14 12:05:02.465625: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-14 12:05:02.488462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771070702.546073     758 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771070702.563065     758 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771070702.618753     758 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

## Optional: make a version of Stardist models with a fixed input size

For macOS users, if the model can run on the GPU using CoreML (I’m not sure all operations are compatible), it may perform better with fixed input shapes.

Source: https://onnxruntime.ai/docs/tutorials/mobile/helpers/make-dynamic-shape-fixed.html  

In [ ]:
!pip install -U onnxruntime onnx


  Using cached onnx-1.20.1-cp312-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: onnx
    Found existing installation: onnx 1.17.0
    Uninstalling onnx-1.17.0:
      Successfully uninstalled onnx-1.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf2onnx 1.16.1 requires protobuf~=3.20, but you have protobuf 6.33.5 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.

In [ ]:

!python -m onnxruntime.tools.make_dynamic_shape_fixed --input_name input --input_shape 1,512,512,3 /content/stardist_onnx_models/he_heavy_augment.onnx /content/stardist_onnx_models/he_heavy_augment_fixed_512.onnx
!python -m onnxruntime.tools.make_dynamic_shape_fixed --input_name input --input_shape 1,512,512,1 /content/stardist_onnx_models/dsb2018_heavy_augment.onnx /content/stardist_onnx_models/dsb2018_heavy_augment_fixed_512.onnx


## Zip the models and download

In [ ]:
!zip -r stardist_onnx_models.zip /content/stardist_onnx_models
from google.colab import files
files.download("stardist_onnx_models.zip")


  adding: content/stardist_onnx_models/ (stored 0%)
  adding: content/stardist_onnx_models/he_heavy_augment_fixed_512.onnx (deflated 8%)
  adding: content/stardist_onnx_models/dsb2018_heavy_augment.onnx (deflated 7%)
  adding: content/stardist_onnx_models/he_heavy_augment.onnx (deflated 8%)
  adding: content/stardist_onnx_models/dsb2018_heavy_augment_fixed_512.onnx (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>